<a href="https://colab.research.google.com/github/Nirjhar100/Major-FakeNews/blob/master/cnn_news_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [0]:
df = pd.read_csv('./clean_data.csv')
print(df.shape)
df.tail()

(16749, 3)


,Unnamed: 0,subreddit,title
16744,9994,0,nh s st coronavirus patient told to stay isola...
16745,9995,0,new hampshire s st coronavirus patient ignores...
16746,9996,0,opinion i just remembered joe biden is fine
16747,9997,0,syria war three year old girl who laughed at b...
16748,9998,0,peeps are the latest food you can stick on you...


In [0]:
df = df.drop(columns=['Unnamed: 0'])

In [0]:
print(df.columns.values)

['subreddit' 'title']


In [0]:
X=df['title']
y=df['subreddit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                     random_state=42)

In [0]:
cv = CountVectorizer()
cv.fit(X_train)
X_train = cv.transform(X_train)
X_test = cv.transform(X_test)
X_train

<12561x17725 sparse matrix of type '<class 'numpy.int64'>'
	with 151961 stored elements in Compressed Sparse Row format>

In [0]:
classifier = MultinomialNB()
classifier.fit(X_train,y_train)
score = classifier.score(X_test,y_test)
print("Accuracy:",score)

Accuracy: 0.8724928366762178


In [0]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(X_train)
train_X = tokenizer.texts_to_sequences(X_train)
test_X = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index)+1
print(vocab_size)
print(X_train[1])
print(train_X[1])

17752
grinning tim cook announces new iphone will no longer be compatible with airpods
[18, 6, 352, 3482, 2073, 3947, 3948, 23, 698, 57]


In [0]:
maxlen = 100
train_X = pad_sequences(train_X, padding = 'post', maxlen=maxlen)
test_X = pad_sequences(test_X, padding = 'post', maxlen = maxlen)
print(train_X[0, :])

[ 17 845 679  11 680 608 445  47   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [0]:
embedding_dim = 50
#input_dim = X_train.shape[1] #number of features
model = Sequential()
#model.add(layers.Dense(64,input_dim = input_dim,activation = 'relu'))
#model.add(layers.Dense(128, activation = 'relu'))

model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                           input_length = maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(10, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 100, 50)           887600    
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 50)                0         
_________________________________________________________________
dense_61 (Dense)             (None, 64)                3264      
_________________________________________________________________
dense_62 (Dense)             (None, 10)                650       
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 11        
Total params: 891,525
Trainable params: 891,525
Non-trainable params: 0
_________________________________________________________________


In [0]:
history=model.fit(train_X,y_train,epochs = 10, verbose = 1, validation_data=(test_X,y_test),batch_size=1024)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 12561 samples, validate on 4188 samples
Epoch 1/10
12561/12561 [==============================] - 1s 52us/step - loss: 0.6900 - accuracy: 0.5273 - val_loss: 0.6840 - val_accuracy: 0.5511
Epoch 2/10
12561/12561 [==============================] - 1s 41us/step - loss: 0.6789 - accuracy: 0.5454 - val_loss: 0.6710 - val_accuracy: 0.5511
Epoch 3/10
12561/12561 [==============================] - 1s 41us/step - loss: 0.6555 - accuracy: 0.5840 - val_loss: 0.6375 - val_accuracy: 0.6843
Epoch 4/10
12561/12561 [==============================] - 1s 43us/step - loss: 0.6007 - accuracy: 0.7788 - val_loss: 0.5724 - val_accuracy: 0.7942
Epoch 5/10
12561/12561 [==============================] - 1s 42us/step - loss: 0.5099 - accuracy: 0.8499 - val_loss: 0.4827 - val_accuracy: 0.8419
Epoch 6/10
12561/12561 [==============================] - 1s 42us/step - loss: 0.3968 - accuracy: 0.8976 - val_loss: 0.3946 - val_accuracy: 0.8553
Epoch 7/10
12561/12561 [==============================] - 1s 42us/ste

In [0]:
loss, accuracy = model.evaluate(train_X, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_X, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9731
Testing Accuracy:  0.8751
